In [1]:
import pandas as pd
import numpy as np
import datetime

# 1. Get raw period

In [10]:
def get_raw_period(y_pred, ts_all_p):
#     new_pred= []
#     for i in range(y_pred):
#         a1 = max(y_pred[i][0],max(y_pred[i][1],max(y_pred[i][2],y_pred[i][3])))
#         for i in range(4):
#             if(a1 == y_pred[i][j]):
#                 new_pred.append(j)
# Input is Array
    ts_all_p = ts_all_p[0: len(y_pred)]
    data = pd.DataFrame(ts_all_p, columns = ['time'])
    data['label'] = pd.DataFrame(y_pred)
    
#     data['label'] = pd.Categorical(y_pred)
    
    label_period = []
    period_list = [[] for i in range(len(LABELS))]
    first = 0
    keep = 0
    
    for i in range(len(data)):
        keep_lb = data.loc[keep]['label'] #keep_lb is label for each loc, then check on each rows.
        if (data.loc[i]['label'] != keep_lb):
            label_period.append([data.loc[keep]['time'], 
                                 data.loc[i-1]['time'],
                                data.loc[i-1]['label']])
            period_list[data.loc[i-1]['label']].append([data.loc[keep]['time'], 
                                                         data.loc[i-1]['time']])
            keep = i
        
    
    table = pd.DataFrame(label_period, columns = ['start','end','label'])
    table['label'].replace({0:'sit', 1:'sleep', 2:'stand', 3:'walk'}, inplace = True)

    table['start'] = pd.to_datetime(table['start'])
    table['end'] = pd.to_datetime(table['end'])

    table['label'] = pd.Categorical(table['label'])
    table['duration'] = table['end'] - table['start']
    
    return table

In [ ]:
def get_raw_period_new(y_pred, timestamp):
    # Input is Dataframe
    df = pd.concat([y_pred, timestamp], axis = 1)
    df.dropna(inplace = True)
    
    LABELS = ['sit', 'sleep', 'stand', 'walk']
    
    label_period = []
    period_list = [[] for i in range(len(LABELS))]
    first = 0
    keep = 0
    
    for i in range(len(df)):
        keep_lb = df.loc[keep]['y_pred'] #keep_lb is label for each loc, then check on each rows.
        if (df.loc[i]['y_pred'] != keep_lb):
            label_period.append([df.loc[keep]['timestamp'], 
                                 df.loc[i-1]['timestamp'],
                                df.loc[i-1]['y_pred'],
                                len(df.iloc[keep:(i-1), 1])])
                                             
            #period_list[df.loc[i-1]['y_pred']].append([df.loc[keep]['timestamp'],df.loc[i-1]['timestamp']])
            keep = i
    #print(len(data))
    
    table = pd.DataFrame(label_period, columns = ['start','end','label', 'duration(timepoint)'])
    table['label'].replace({0:'sit', 1:'sleep', 2:'stand', 3:'walk'}, inplace = True)

    table['start'] = pd.to_datetime(table['start'])
    table['end'] = pd.to_datetime(table['end'])

    table['label'] = pd.Categorical(table['label'])
    #table['duration'] = table['end'] - table['start']
    
    return table

# 2. Get 24 hr. period & count changing

In [9]:
def count_chg(status):
    n = 0
    status2 = status.reset_index()
    for i in range(len(status2)):
        if i < len(status)-1:
            if status2.loc[i]['status'] != status2.loc[i+1]['status']:
                n = n+1
                #print(n)
    return n

In [ ]:
def count_action(time):
    
    sit = len(time[time['status'] == 'sit'])
    stand = len(time[time['status'] == 'stand'])
    walk = len(time[time['status'] == 'walk'])
    sleep = len(time[time['status'] == 'sleep'])

    action = [sit, stand, walk, sleep]
    
    return action

In [ ]:
def action_duration(time):
    sit = time[time['status'] == 'sit']
    stand = time[time['status'] == 'stand']
    walk = time[time['status'] == 'walk']
    sleep = time[time['status'] == 'sleep']
    
    du_sit = sit['duration'].sum()
    du_stand = stand['duration'].sum()
    du_walk = walk['duration'].sum()
    du_sleep = sleep['duration'].sum()
    
    action_duration = [du_sit, du_stand, du_walk, du_sleep]
    
    return action_duration

In [ ]:
def get_24_period(label_period):

#Build Empty DataFrame 
    period = {'start':['00:00:00', '01:00:00', '02:00:00', '03:00:00', '04:00:00', '05:00:00', '06:00:00', '07:00:00', '08:00:00','09:00:00',
                      '10:00:00','11:00:00','12:00:00','13:00:00','14:00:00','15:00:00','16:00:00','17:00:00','18:00:00','19:00:00','20:00:00',
                       '21:00:00','22:00:00','23:00:00'],
             'end':['00:59:00', '01:59:00', '02:59:00', '03:59:00', '04:59:00', '05:59:00', '06:59:00', '07:59:00', '08:59:00','09:59:00',
                      '10:59:00','11:59:00','12:59:00','13:59:00','14:59:00','15:59:00','16:59:00','17:59:00','18:59:00','19:59:00','20:59:00',
                       '21:59:00','22:59:00','23:59:00'],
             'Status Change': 0}
    period = pd.DataFrame(period)
    period['duration'] = 0
    
#Back up raw data (table = raw data)
    status = pd.DataFrame(label_period['start'])
    status['end'] = pd.DataFrame(label_period['end'])
    status['status'] = label_period['label']
    status['status'] = pd.Categorical(status['status'])
    status['duration'] = label_period['duration']
    status['start'] = [datetime.datetime.time(d) for d in status['start']]# extracting time from timestamp
    status['end'] = [datetime.datetime.time(d) for d in status['end']]

#select 24 hour period to 24 objects 
    first = status[(status['start'] >= datetime.time(00,00,00)) & (status['end'] <= datetime.time(00,59,00))]
    second = status[(status['start'] >= datetime.time(1,00,00)) & (status['end'] <= datetime.time(1,59,00))]
    third = status[(status['start'] >= datetime.time(2,00,00)) & (status['end'] <= datetime.time(2,59,00))]
    forth = status[(status['start'] >= datetime.time(3,00,00)) & (status['end'] <= datetime.time(3,59,00))]
    fifth = status[(status['start'] >= datetime.time(4,00,00)) & (status['end'] <= datetime.time(4,59,00))]
    sixth = status[(status['start'] >= datetime.time(5,00,00)) & (status['end'] <= datetime.time(5,59,00))]
    seventh = status[(status['start'] >= datetime.time(6,00,00)) & (status['end'] <= datetime.time(6,59,00))]
    eight = status[(status['start'] >= datetime.time(7,00,00)) & (status['end'] <= datetime.time(7,59,00))]
    ninth = status[(status['start'] >= datetime.time(8,00,00)) & (status['end'] <= datetime.time(8,59,00))]
    tenth = status[(status['start'] >= datetime.time(9,00,00)) & (status['end'] <= datetime.time(9,59,00))]
    eleventh = status[(status['start'] >= datetime.time(10,00,00)) & (status['end'] <= datetime.time(10,59,00))]
    twelth = status[(status['start'] >= datetime.time(11,00,00)) & (status['end'] <= datetime.time(11,59,00))]
    thirteenth = status[(status['start'] >= datetime.time(12,00,00)) & (status['end'] <= datetime.time(12,59,00))]
    fourteenth = status[(status['start'] >= datetime.time(13,00,00)) & (status['end'] <= datetime.time(13,59,00))]
    fifteenth = status[(status['start'] >= datetime.time(14,00,00)) & (status['end'] <= datetime.time(14,59,00))]
    sixteenth = status[(status['start'] >= datetime.time(15,00,00)) & (status['end'] <= datetime.time(15,59,00))]
    seventeenth = status[(status['start'] >= datetime.time(16,00,00)) & (status['end'] <= datetime.time(16,59,00))]
    eighteenth = status[(status['start'] >= datetime.time(17,00,00)) & (status['end'] <= datetime.time(17,59,00))]
    nineteenth = status[(status['start'] >= datetime.time(18,00,00)) & (status['end'] <= datetime.time(18,59,00))]
    twentieth = status[(status['start'] >= datetime.time(19,00,00)) & (status['end'] <= datetime.time(19,59,00))]
    twentieoneth = status[(status['start'] >= datetime.time(20,00,00)) & (status['end'] <= datetime.time(20,59,00))]
    twentietwoth = status[(status['start'] >= datetime.time(21,00,00)) & (status['end'] <= datetime.time(21,59,00))]
    twentiethreeth = status[(status['start'] >= datetime.time(22,00,00)) & (status['end'] <= datetime.time(22,59,00))]
    twentiefourth = status[(status['start'] >= datetime.time(23,00,00)) & (status['end'] <= datetime.time(23,59,00))]
    

    #Apply function to count each selected period

    period.iloc[0,2] = count_chg(first)
    period.iloc[1,2] = count_chg(second)
    period.iloc[2,2] = count_chg(third)
    period.iloc[3,2] = count_chg(forth)
    period.iloc[4,2] = count_chg(fifth)
    period.iloc[5,2] = count_chg(sixth)
    period.iloc[6,2] = count_chg(seventh)
    period.iloc[7,2] = count_chg(eight)
    period.iloc[8,2] = count_chg(ninth)
    period.iloc[9,2] = count_chg(tenth)
    period.iloc[10,2] = count_chg(eleventh)
    period.iloc[11,2] = count_chg(twelth)
    period.iloc[12,2] = count_chg(thirteenth)
    period.iloc[13,2] = count_chg(fourteenth)
    period.iloc[14,2] = count_chg(fifteenth)
    period.iloc[15,2] = count_chg(sixteenth)
    period.iloc[16,2] = count_chg(seventeenth)
    period.iloc[17,2] = count_chg(eighteenth)
    period.iloc[18,2] = count_chg(nineteenth)
    period.iloc[19,2] = count_chg(twentieth)
    period.iloc[20,2] = count_chg(twentieoneth)
    period.iloc[21,2] = count_chg(twentietwoth)
    period.iloc[22,2] = count_chg(twentiethreeth)
    period.iloc[23,2] = count_chg(twentiefourth)
    
#sum up the duration

    period.iloc[0,3] = first['duration'].sum()
    period.iloc[1,3] = second['duration'].sum()
    period.iloc[2,3] = third['duration'].sum()
    period.iloc[3,3] = forth['duration'].sum()
    period.iloc[4,3] = fifth['duration'].sum()
    period.iloc[5,3] = sixth['duration'].sum()
    period.iloc[6,3] = seventh['duration'].sum()
    period.iloc[7,3] = eight['duration'].sum()
    period.iloc[8,3] = ninth['duration'].sum()
    period.iloc[9,3] = tenth['duration'].sum()
    period.iloc[10,3] = eleventh['duration'].sum()
    period.iloc[11,3] = twelth['duration'].sum()
    period.iloc[12,3] = thirteenth['duration'].sum()
    period.iloc[13,3] = fourteenth['duration'].sum()
    period.iloc[14,3] = fifteenth['duration'].sum()
    period.iloc[15,3] = sixteenth['duration'].sum()
    period.iloc[16,3] = seventeenth['duration'].sum()
    period.iloc[17,3] = eighteenth['duration'].sum()
    period.iloc[18,3] = nineteenth['duration'].sum()
    period.iloc[19,3] = twentieth['duration'].sum()
    period.iloc[20,3] = twentieoneth['duration'].sum()
    period.iloc[21,3] = twentietwoth['duration'].sum()
    period.iloc[22,3] = twentiethreeth['duration'].sum()
    period.iloc[23,3] = twentiefourth['duration'].sum()

#count action
    period.iloc[0,[4,5,6,7]] = count_action(first)
    period.iloc[1,[4,5,6,7]] = count_action(second)
    period.iloc[2,[4,5,6,7]] = count_action(third)
    period.iloc[3,[4,5,6,7]] = count_action(forth)
    period.iloc[4,[4,5,6,7]] = count_action(fifth)
    period.iloc[5,[4,5,6,7]] = count_action(sixth)
    period.iloc[6,[4,5,6,7]] = count_action(seventh)
    period.iloc[7,[4,5,6,7]] = count_action(eight)
    period.iloc[8,[4,5,6,7]] = count_action(ninth)
    period.iloc[9,[4,5,6,7]] = count_action(tenth)
    period.iloc[10,[4,5,6,7]] = count_action(eleventh)
    period.iloc[11,[4,5,6,7]] = count_action(twelth)
    period.iloc[12,[4,5,6,7]] = count_action(thirteenth)
    period.iloc[13,[4,5,6,7]] = count_action(fourteenth)
    period.iloc[14,[4,5,6,7]] = count_action(fifteenth)
    period.iloc[15,[4,5,6,7]] = count_action(sixteenth)
    period.iloc[16,[4,5,6,7]] = count_action(seventeenth)
    period.iloc[17,[4,5,6,7]] = count_action(eighteenth)
    period.iloc[18,[4,5,6,7]] = count_action(nineteenth)
    period.iloc[19,[4,5,6,7]] = count_action(twentieth)
    period.iloc[20,[4,5,6,7]] = count_action(twentieoneth)
    period.iloc[21,[4,5,6,7]] = count_action(twentietwoth)
    period.iloc[22,[4,5,6,7]] = count_action(twentiethreeth)
    period.iloc[23,[4,5,6,7]] = count_action(twentiefourth)
    
#Duration each action
    period.iloc[0,[8,9,10,11]] = action_duration(first)
    period.iloc[1,[8,9,10,11]] = action_duration(second)
    period.iloc[2,[8,9,10,11]] = action_duration(third)
    period.iloc[3,[8,9,10,11]] = action_duration(forth)
    period.iloc[4,[8,9,10,11]] = action_duration(fifth)
    period.iloc[5,[8,9,10,11]] = action_duration(sixth)
    period.iloc[6,[8,9,10,11]] = action_duration(seventh)
    period.iloc[7,[8,9,10,11]] = action_duration(eight)
    period.iloc[8,[8,9,10,11]] = action_duration(ninth)
    period.iloc[9,[8,9,10,11]] = action_duration(tenth)
    period.iloc[10,[8,9,10,11]] = action_duration(eleventh)
    period.iloc[11,[8,9,10,11]] = action_duration(twelth)
    period.iloc[12,[8,9,10,11]] = action_duration(thirteenth)
    period.iloc[13,[8,9,10,11]] = action_duration(fourteenth)
    period.iloc[14,[8,9,10,11]] = action_duration(fifteenth)
    period.iloc[15,[8,9,10,11]] = action_duration(sixteenth)
    period.iloc[16,[8,9,10,11]] = action_duration(seventeenth)
    period.iloc[17,[8,9,10,11]] = action_duration(eighteenth)
    period.iloc[18,[8,9,10,11]] = action_duration(nineteenth)
    period.iloc[19,[8,9,10,11]] = action_duration(twentieth)
    period.iloc[20,[8,9,10,11]] = action_duration(twentieoneth)
    period.iloc[21,[8,9,10,11]] = action_duration(twentietwoth)
    period.iloc[22,[8,9,10,11]] = action_duration(twentiethreeth)
    period.iloc[23,[8,9,10,11]] = action_duration(twentiefourth)
    return period

# 3. Get 4 period and transition score

In [13]:
def get_4_period(period):

#Build empty dataframe
    period_four = {'start':[datetime.time(6,0,0), datetime.time(12,0,0), datetime.time(18,0,0), datetime.time(0,0,0)],
                   'end':[datetime.time(11,59,0), datetime.time(17,59,0), datetime.time(23,59,0), datetime.time(5,59,0)],
                   'Status Change': 0,
                   'duration': 0,
                   'duration(minutes)' : 0,
                   't/m' : 0}
    period_four = pd.DataFrame(period_four)

#Sum up Transition Change and Duration
    period_four.iloc[0, 2] = np.sum(period.iloc[6:11, 2])
    period_four.iloc[0, 3] = np.sum(period.iloc[6:11, 3])

    period_four.iloc[1, 2] = np.sum(period.iloc[12:17, 2])
    period_four.iloc[1, 3] = np.sum(period.iloc[12:17, 3])

    period_four.iloc[2, 2] = np.sum(period.iloc[18:23, 2])
    period_four.iloc[2, 3] = np.sum(period.iloc[18:23, 3])

    period_four.iloc[3, 2] = np.sum(period.iloc[0:5, 2])
    period_four.iloc[3, 3] = np.sum(period.iloc[0:5, 3])
    
    period_four['duration'] = pd.to_timedelta(period_four['duration'])
    
#Execute transition score (per minutes)
    period_four.iloc[0,4]= period_four.loc[0]['duration'].seconds/60
    period_four.iloc[1,4] = period_four.loc[1]['duration'].seconds/60
    period_four.iloc[2,4] = period_four.loc[2]['duration'].seconds/60
    period_four.iloc[3,4] = period_four.loc[3]['duration'].seconds/60
    
    period_four.iloc[0, 5] = period_four.loc[0]['Status Change']/period_four.loc[0]['duration(minutes)']
    period_four.iloc[1, 5] = period_four.loc[1]['Status Change']/period_four.loc[1]['duration(minutes)']
    period_four.iloc[2, 5] = period_four.loc[2]['Status Change']/period_four.loc[2]['duration(minutes)']
    period_four.iloc[3, 5] = period_four.loc[3]['Status Change']/period_four.loc[3]['duration(minutes)']
    
    #Action Count
    period_four.iloc[0, [5,6,7,8]] = np.sum(period.iloc[6:12, [4,5,6,7]])
    period_four.iloc[1, [5,6,7,8]] = np.sum(period.iloc[12:18, [4,5,6,7]])
    period_four.iloc[2, [5,6,7,8]] = np.sum(period.iloc[18:24, [4,5,6,7]])
    period_four.iloc[3, [5,6,7,8]] = np.sum(period.iloc[0:6, [4,5,6,7]])
    period_four.iloc[4, [5,6,7,8]] = np.sum(period.iloc[0:24, [4,5,6,7]])

#Sum Duration (Minutes)
    period_four.iloc[0, [9,10,11,12]] = np.sum(period.iloc[6:12, [8,9,10,11]])/60
    period_four.iloc[1, [9,10,11,12]] = np.sum(period.iloc[12:18, [8,9,10,11]])/60
    period_four.iloc[2, [9,10,11,12]] = np.sum(period.iloc[18:24, [8,9,10,11]])/60
    period_four.iloc[3, [9,10,11,12]] = np.sum(period.iloc[0:6, [8,9,10,11]])/60
    period_four.iloc[4, [9,10,11,12]] = np.sum(period.iloc[0:24, [8,9,10,11]])/60

    return period_four

# 4. Get total transition score

In [17]:
def get_summary(y_pred, time):
    raw = get_raw_period(y_pred, ts_all_cut)
    period24 = get_24_period(raw)
    summary = get_4_period(period24)
    return summary